In [ ]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
# 데이터셋 업로드 
from google.colab import files
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False


In [ ]:

# 데이터 로드 
uploaded = files.upload() 

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn]))) 
data = pd.read_csv('/content/integrated_data.csv')
data['new_Date'] = pd.to_datetime(data['date'])
print(data.head())
print('\n')
print(data.info())
print('\n')
#첫번째 컬럼을 날짜 컬럼으로 변경 
print(type(data['new_Date'][0]))
data.drop('date', axis = 1, inplace=True)
data.set_index('new_Date', inplace=True)
print(data.head())
print('\n')
print(data.info())


In [ ]:
# 5열까지 출력 
data.head(5)


In [ ]:
#data['house_loan'].plot()
plt.plot(data['house_loan'])
plt.show()
plt.plot(data['consumer_price'])
plt.show()
plt.plot(data['building_space'])
plt.show()
plt.plot(data['interest_idx'])
plt.show()
plt.plot(data['housing_price'])
plt.show()


In [ ]:
#전체 데이터 출력 
data.head()
economic_stat = ['consumer_price','house_loan','building_space','interest_idx','housing_price']
features = data[economic_stat]
#features.index = df['date']
print(features.head(156))
print(features.values)
features.plot(subplots=True)
plt.show()

In [ ]:
#훈련데이터 분할 
#2008-01-01 ~ 2018-01-01 120개월을훈련데이터로 분할 
# 각 데이터 원시값-평균/표준편차 => 데이터셋 표준화 
TRAIN_SPLIT =120
tf.random.set_seed(13)
dataset = features.values
#print(dataset)
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)#각열에 대한120인덱스 까지의  평균값 
#print(data_mean)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)# 각 열에 대한 표준편차차
print(data_std)
dataset = (dataset-data_mean)/data_std
print(dataset)

In [ ]:
#신경망 모델 훈련에 사용한 특정시간 윈도우 데이터 확보를 위한 함수 
#history: 과거데이터 크기
#target size 모델이 미래를 예측할 크기 
# step값이 주어지면 step에 맞게 데이터를 샘플링 
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []          #
    labels = []
    start_index = start_index + history_size# 0 + 24
    if end_index is None:
        end_index = len(dataset) - target_size
        #마지막 인덱스 값이 지정되지 않으면 학습 데이터셋 크기에서 타겟데이터(예측사이즈 6으로 지정됨)를 뺌 
    for i in range(start_index, end_index): # (이전예측인덱스 사이즈)720~30000 
        indices = range(i - history_size, i, step)# 1번째 루프 : 720-720 =0 ~ 720 까지 6스텝 
        # 0,27,3 -> 0,3,6,9,12,15,18,21,24(9개) +레이블 25
        print(indices)
        
        """
        예제 데이터: 10분단위 온도데이터 
        훈련데이터 스플릿 30000
        과거데이터: 5일 
        예측할 데이터: 72 -> 720분-> 12시간 후 데이터 예측 
        싱글스텝:60(1시간)
        훈련데이터 포맷
        0에서 5일(720분       싱글스텝(샘플링 간격):60(1시간)
        훈련데이터 포맷
        0분에서 5일(7200분->120시간->5일)동안 1시간 마다의 예측데이터를 포함한 데이터셋 
        range(0, 720, 6)
        10분에서 7210분 동안 1시간마다 데이터 
        range(1, 721, 6)
        range(2, 722, 6)
        range(3, 723, 6)
        range(4, 724, 6)
  
        range(11, 731, 6)
        ...
        """
        #메소드 안에서 생성된 리스트에 표준화된 데이터셋을 0~113 인덱스까지 때ㅂ려박음 
        data.append(dataset[indices])#
        #single step가 true ㅇ이면 
        # 스텝크기가 0일때는 start~endinx 까지 박아넣음 
        # 학습에 필요한 타겟데이터
        
        if single_step:
            labels.append(target[i + target_size])#true이면 720 + 72(5일+12시간), 721 + 72 ... 
            print('타겟 데이터')
            print(target[i + target_size])
        else:
            labels.append(target[i:i + target_size])#720 ~ 720+72, 721 ~ 721+72, ...  
    return np.array(data), np.array(labels)

In [ ]:
past_history = 24
future_target = 3
STEP = 2
"""
데이터 샘플링 
현재 보유중인 데이터: 월단위 
데이터 스플릿 120개월, 총 데이터 156 13년 20081.1 2020121
샘플링 단위 1개월 
과거 2년(24개월) 단위로 3개월 씩 샘플링 
타겟 1개월 예측 
range(0, 24, 1) 
range(1, 25, 1)
range(2, 26, 1)
range(3, 27, 3)
range(4, 28, 3)
range(5, 29, 3)
range(6, 30, 3)
range(7, 31, 3)
range(8, 32, 3)
학습 타겟 데이터: 훈련데이터가 매 샘플마다 저장될 때 레이블 데이터는 
해당 샘플의 마지막 인덱스+1(24일 때 25, 바로 다음달 ) 한 값을 타겟데이터로 저장 
"""

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 4], 0, TRAIN_SPLIT, past_history, future_target, STEP, single_step=True)#0 ~ 30000
print('-----------------------------------')

# 예측해야할 데이터셋  11개 윈도우 120 ~ 154
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 4], TRAIN_SPLIT, None, past_history, future_target, STEP, single_step=True)#30000 ~ end

print('Single window of past history : {}'.format(x_train_single[0].shape))
print(y_train_single)
print('================================================================')
print('Single window of past history : {}'.format(x_val_single[0].shape))
print(x_val_single)
print('y')
print(y_val_single)
# 처음부터 약2년까지의  3개월 단위 표준화 데이터셋 5개의 컬럼, 8개 행.

# 훈련데이터 차원 130,8,5..?

In [ ]:
BATCH_SIZE = 96##144 #128#256
#Batch는 일괄 처리되는 작업의 양이다. 위에서 설명된 data_size를 한번에 처리하는 갯수를 의미하며, 
#batch크기에 의해 weight 변화가 일어나며 batch크기 단위로 data loading 함수도 구현이 가능하다. 
#주의할점은 data_size/batch_size일때 나머지가 없어야 한다.
BUFFER_SIZE = 4096 #데이터 14000
EVALUATION_INTERVAL = 9
EPOCHS = 100
# 입력된 텐서로부터 slices를 생성한다. 
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
count =0
for element in train_data_single:
  print("훈련데이터 슬라이스")
  count = count+1
  print(element)
print(count)  
  # 입력된 텐서로부터 slices를 생성한다. 무기한 반복
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
# 데이터 캐싱, 훈련데이터 셔플 
val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single)) #11,9,5 11,0
for element in val_data_single:
  print("평가데이터 슬라이스")
  print(element)
print("이;거뭕ㄷ데")
print(val_data_single)
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

#for element in val_data_single:
#  print(element)
print(x_train_single.shape) #96,25,5
print(x_train_single.shape[-2:])

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
single_step_model = tf.keras.models.Sequential()
#입력데이터 형식: 소비자물가지수, 가계대출금, 금리, 건축허가면적, 주택가격지수  입력형식 [8,5]
single_step_model.add(tf.keras.layers.LSTM(20, activation='tanh', input_shape=x_train_single.shape[-2:],dropout=0.5)) 
#출력값 1
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.Adam(), loss = root_mean_squared_error)# 에러 절댓값 평균ingle_step_model.add(tf.keras.layers.Dense(1))
    
#print(len(single_step_model.layers))#2
#훈련시작 배치크기 50회 학습, 

#for x, y in val_data_single.take(3):
#  print('이게뭔데 그래서')
#  print(single_step_model.predict(x).shape)
#  print(single_step_model.predict(x))
        
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=3)
#steps_per_epoch [트레이닝데이터수/배치사이즈]
#validation_steps validation data수/배치사이즈


print(single_step_history)
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()

    plt.show()

plot_train_history(single_step_history,
                   'Single Step Training and Validation Loss')

In [ ]:
def create_time_steps(length):
    return list(range(-length, 0))

 

def show_plot(plot_data, delta, title):
    labels = ['History', 'True Future', 'Model Prediction']
    marker = ['.-', 'rx', 'go']
    time_steps = create_time_steps(plot_data[0].shape[0])#24
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.axis('auto')
    plt.xlim([time_steps[0], future*2])#(future+5)*2])
    plt.xlabel('Time-Step')
    return plt
    
print("val_data_single.take(3)")
print(val_data_single.take(3))
for x, y in val_data_single.take(3):
  print('이게뭔데 그래서')
  print('x')# 11,24,5 
  print(x)
  print('y')
  print(y)
  print('---------------------------')

  count=0;
for x, y in val_data_single.take(1):
  # 매개변수가 리스트임 평가데이터가 x는 11,24,5 의 shape을 가지고 있는데 거기서 0,8,4 즉 8개월 동안의 주택가격지수
    print('==============') 
    print(x.shape) 
    print('====////////////') 
    for window in range(3):
      print([x[window][:, 4].numpy(), y[window].numpy(),single_step_model.predict(x)[window]])
      plot = show_plot([x[window][:, 4].numpy(), y[window].numpy(),single_step_model.predict(x)[window]], 1, 'Single Step Prediction')
      plot.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Pridicting All time Housing price")
plt.plot(data['housing_price'])
pridict = []
for x,y in train_data_single.take(1):
  print(single_step_model.predict(x).shape)
  pridict = single_step_model.predict(x)
  plt.plot(single_step_model.predict(x))
  #for window in range(96):
  #plt.show()
print("hello")
print(pridict)
plt.plot(pridict)